In [1]:
from FunctionSet import *
# The NRPy_param_funcs module sets up global structures that manage free parameters within NRPy+
import NRPy_param_funcs as par 
# The indexedexp module defines various functions for defining and managing indexed quantities like tensors and pseudotensors
import indexedexp as ixp
# The grid module defines various parameters related to a numerical grid or the dimensionality of indexed expressions
# For example, it declares the parameter DIM, which specifies the dimensionality of the indexed expression
import grid as gri
from outputC import *
import numpy as np

import sympy as sp
from sympy import symbols, IndexedBase, Indexed, Idx, preorder_traversal
from sympy.matrices import *

Declaring indexed variables

In [2]:
Nx, Ny, Nz= symbols('Nx Ny Nz', integer=True)
i = Idx('i', Nx)
j = Idx('j', Ny)
k = Idx('k', Nz)
name = Idx('name')
dx, dy, dz = symbols('dx dy dz')

Declaring variable names to be replaced later in AMReX code output.

In [3]:
var_alpha = ['alpha']

alpha = IndexedBase('state_fab'+var_alpha[0])[i,j,k,var_alpha[0]]

var_beta = ['betax','betay','betaz']

betaL = ixp.zerorank1(DIM=3)

for itr in range(len(directions)):
    betaL[itr] = IndexedBase('state_fab'+var_beta[itr])[i,j,k,var_beta[itr]]
    
betaL = np.array(betaL)

var_gam = [['gxx','gxy','gxz'],['gyx','gyy','gyz'],['gzx','gzy','gzz']]

gamLL = ixp.declarerank2('g','sym01')
for itr1 in range(len(directions)):
    for itr2 in range(len(directions)):
        gamLL[itr1][itr2] = IndexedBase('state_fab'+var_gam[itr1][itr2])[i,j,k,var_gam[itr1][itr2]]

gamLL = np.array(gamLL)

var_gaminv = [['ginvxx','ginvxy','ginvxz'],['ginvyx','ginvyy','ginvyz'],['ginvzx','ginvzy','ginvzz']]
        
gamUU = ixp.declarerank2('ginv','sym01')
for itr1 in range(len(directions)):
    for itr2 in range(len(directions)):
        gamUU[itr1][itr2] = IndexedBase('state_fab'+var_gaminv[itr1][itr2])[i,j,k,var_gaminv[itr1][itr2]]
        
gamUU = np.array(gamUU)

var_K = [['Kxx','Kxy','Kxz'],['Kyx','Kyy','Kyz'],['Kzx','Kzy','Kzz'],]
        
KLL = ixp.declarerank2('K','sym01')
for itr1 in range(len(directions)):
    for itr2 in range(len(directions)):
        KLL[itr1][itr2] = IndexedBase('state_fab'+var_K[itr1][itr2])[i,j,k,var_K[itr1][itr2]]

KLL = np.array(KLL)



In [4]:
betaU = ixp.zerorank1(3)
for itri in range(len(directions)):
    for itrj in range(len(directions)):
        betaU[itri] = gamUU[itri][itrj]*betaL[itrj]

Organizing variable names into a complete list.

In [5]:
var_names = var_beta
for iter1 in var_gam:
    var_names.extend(iter1)
for iter2 in var_gaminv:
    var_names.extend(iter2)
for iter3 in var_K:
    var_names.extend(iter3)
var_names.insert(0,var_alpha[0])
print(var_names)

['alpha', 'betax', 'betay', 'betaz', 'gxx', 'gxy', 'gxz', 'gyx', 'gyy', 'gyz', 'gzx', 'gzy', 'gzz', 'ginvxx', 'ginvxy', 'ginvxz', 'ginvyx', 'ginvyy', 'ginvyz', 'ginvzx', 'ginvzy', 'ginvzz', 'Kxx', 'Kxy', 'Kxz', 'Kyx', 'Kyy', 'Kyz', 'Kzx', 'Kzy', 'Kzz']


Definging Christoffel symbols.

In [6]:
DcgLLL = ixp.zerorank3(3)
GammaULL = ixp.zerorank3(3)

for itri in range(len(directions)):
    for itrj in range(len(directions)):
        for itrk in range(len(directions)):
            DcgLLL[itri][itrj][itrk] = Dc(gamLL[itri][itrj],directions[itrk])
            
for itri in range(len(directions)):
    for itrj in range(len(directions)):
        for itrk in range(len(directions)):
            for itrl in range(len(directions)):
                GammaULL[itri][itrj][itrk] += 1/2*gamUU[itri][itrl]*(DcgLLL[itrj][itrl][itrk]+DcgLLL[itrk][itrl][itrj]-DcgLLL[itrj][itrk][itrl])

Defining derivatives of Christoffel symbols. This will take a few seconds.

In [8]:
divGammaLULL = ixp.zerorank4(3)

for itri in range(len(directions)):
    for itrj in range(len(directions)):
        for itrk in range(len(directions)):
            for itrl in range(len(directions)):
                divGammaLULL[itrl][itri][itrj][itrk] = Dc(GammaULL[itri][itrj][itrk],directions[itrl])
    

Defining spatial Ricci tensor.

In [9]:
RLL = ixp.zerorank2(3)
for itri in range(len(directions)):
    for itrj in range(len(directions)):
        for itrk in range(len(directions)):
            RLL[itri][itrj] += divGammaLULL[itrk][itrk][itri][itrj]-divGammaLULL[itrj][itrk][itri][itrk]
            for itrl in range(len(directions)):
                RLL[itri][itrj] += GammaULL[itrk][itrl][itrk]*GammaULL[itrl][itri][itrj]-GammaULL[itrk][itrl][itri]*GammaULL[itrl][itrj][itrk]

In [10]:
DDalpha = ixp.zerorank2(3) 
for itri in range(len(directions)):
    for itrj in range(len(directions)):
        DDalpha[itri][itrj]= Dc(Dc(alpha,directions[itri]),directions[itrj])

In [11]:
TrK = 0
for itri in range(len(directions)):
    for itrj in range(len(directions)):
        TrK += gamUU[itri][itrj]*KLL[itri][itrj]

In [12]:
RHS_KLL = ixp.zerorank2(3)
KKLL= ixp.zerorank2(3)
for itri in range(len(directions)):
    for itrj in range(len(directions)):
        for itrk in range(len(directions)):
            for itrl in range(len(directions)):
                KKLL[itri][itrj] = KLL[itri][itrk]*gamUU[itrk][itrl]*KLL[itrl][itrj]
for itri in range(len(directions)):
    for itrj in range(len(directions)):
        RHS_KLL[itri][itrj] += alpha*(RLL[itri][itrj]-2*KKLL[itri][itrj]+TrK*KLL[itri][itrj])-DDalpha[itri][itrj]
        for itrk in range(len(directions)):
            RHS_KLL[itri][itrj] += betaU[itrk]*Dc(KLL[itri][itrj],directions[0])+KLL[itri][itrk]*Dc(betaU[itrk],directions[itrj])+KLL[itrk][itrj]*Dc(betaU[itrk],directions[itri])
            

One of six examples of AMReXcode output for K_ij.

In [13]:
AMReXcode(RLL[0][1],var_names,True, var_K[0][1])

'rhs_fab(i, j, k, Idx::Kxy) = (0.5*(-(-state_fab(i, j - 1, k, Idx::gxx) + state_fab(i, j + 1, k, Idx::gxx))/(2*dx[1]) + (-state_fab(i - 1, j, k, Idx::gxy) + state_fab(i + 1, j, k, Idx::gxy))/dx[0])*state_fab(i, j, k, Idx::ginvxy) + 0.5*(-(-state_fab(i, j, k - 1, Idx::gxx) + state_fab(i, j, k + 1, Idx::gxx))/(2*dx[2]) + (-state_fab(i - 1, j, k, Idx::gxz) + state_fab(i + 1, j, k, Idx::gxz))/dx[0])*state_fab(i, j, k, Idx::ginvxz) + 0.25*(-state_fab(i - 1, j, k, Idx::gxx) + state_fab(i + 1, j, k, Idx::gxx))*state_fab(i, j, k, Idx::ginvxx)/dx[0])*(0.5*((-state_fab(i, j - 1, k, Idx::gxx) + state_fab(i, j + 1, k, Idx::gxx))/(2*dx[1]) - (-state_fab(i - 1, j, k, Idx::gxy) + state_fab(i + 1, j, k, Idx::gxy))/(2*dx[0]) + (-state_fab(i - 1, j, k, Idx::gyx) + state_fab(i + 1, j, k, Idx::gyx))/(2*dx[0]))*state_fab(i, j, k, Idx::ginvxx) + 0.5*(-(-state_fab(i, j, k - 1, Idx::gxy) + state_fab(i, j, k + 1, Idx::gxy))/(2*dx[2]) + (-state_fab(i, j - 1, k, Idx::gxz) + state_fab(i, j + 1, k, Idx::gxz))/(2*d